## В данном ноутбуке происходит разбор способности различных моделей различать различные публикации на определенные классы.

#### Сравнение будет происходить на датасете cora

---
 - 1-ая модель будет представлять из себя обычную модель бустинга
 - 2-я модель будет представлять из себя нейронную сеть с использованием граовой нейронной сети

### Сначала скачаем датасет.
Его можно скачать как с  https://graphsandnetworks.com/the-cora-dataset/

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

In [2]:
### Считывание данных для ребер графа
edgelist = pd.read_csv('cora_dataset/cora.cites', sep='\t', header=None, names=['target', 'source'])

In [3]:
### Считывание данных для узлов графв
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_csv('cora_dataset/cora.content', sep='\t', header=None, names=column_names)

In [4]:
### Посмотрим на данные, содержащиеся в информации об узлах

In [5]:
node_data

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,subject
31336,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1061127,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
1106406,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
13195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
37879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
1128977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
1128978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
117328,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based


In [6]:
# Закодируем категорию

In [7]:
le = LabelEncoder()
le.fit(node_data.subject)
node_data.subject = le.transform(node_data.subject)

In [8]:
le.classes_

array(['Case_Based', 'Genetic_Algorithms', 'Neural_Networks',
       'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning',
       'Theory'], dtype=object)

Не зная графовую структуру цитирования публикаций, о ее содержании можно судить только по тексту. Однако некоторые темы и топики очень близки друг к другу по смыслу и текстовая информация может быть не очень очевидна для текста. Это можно продемонстровать на примере нескольких моделей.

In [9]:
### Для дальнейшей удобной работы с графами преименуем узлы

In [10]:
def node_dict(nodes_nums):
    values = list(range(len(nodes_nums)))
    return dict(zip(nodes_nums, values))

In [11]:
edgelist.target = edgelist.target.map(node_dict(node_data.index))
edgelist.source = edgelist.source.map(node_dict(node_data.index))

In [12]:
node_data.index = node_data.index.map(node_dict(node_data.index))

В первую очередь разобьем все существующие узлы на трейн и тест. 25% будут с неизвестным топиком

In [13]:
train_nodes, test_nodes = train_test_split(node_data.index, test_size=0.25, shuffle=True, random_state=42)

In [14]:
train_X, train_y, test_X, test_y = node_data.loc[train_nodes].drop(columns='subject'), node_data.loc[train_nodes].subject, node_data.loc[test_nodes].drop(columns='subject'), node_data.loc[test_nodes].subject

In [15]:
model = XGBClassifier(max_depth=5, n_estimators=200)

In [16]:
model.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [17]:
preds = model.predict(test_X)

In [18]:
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.74      0.70      0.72        77
           1       0.84      0.83      0.83       105
           2       0.77      0.87      0.82       214
           3       0.80      0.75      0.78       101
           4       0.76      0.84      0.80        44
           5       0.86      0.38      0.53        47
           6       0.67      0.69      0.68        89

    accuracy                           0.77       677
   macro avg       0.78      0.72      0.74       677
weighted avg       0.77      0.77      0.76       677



In [19]:
### Построим графовую нейронную сеть для решения аналогичной задачи

In [74]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [110]:
edge_index = torch.tensor(edgelist.values.transpose(), dtype=torch.long)

In [111]:
features = node_data.reset_index().sort_values('index', ascending=True).drop(columns=['index', 'subject']).values
features = torch.tensor(features, dtype=torch.float)

In [112]:
target = torch.tensor(node_data.reset_index().sort_values('index', ascending=True).subject.values, dtype=torch.long)

In [134]:
data = Data(x=features, edge_index=edge_index, y = target)

In [135]:
data

Data(x=[2708, 1433], edge_index=[2, 5429], y=[2708])

In [129]:
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, GATv2Conv
from torch import nn

In [136]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATv2Conv(1433, 200)
        self.conv2 = GATv2Conv(200, 7)
        self.relu = nn.ReLU()
    def forward(self, x):
        x, edge_index = x.x, x.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [137]:
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [138]:
train_mask = torch.tensor(node_data.index.isin(train_nodes), dtype=torch.bool)

In [139]:
for epoch in tqdm(range(200)):
    optimizer.zero_grad()
    preds = model(data)[train_mask]
    reals = data.y[train_mask]
    loss = torch.nn.CrossEntropyLoss()(preds, reals)
    loss.backward()
    optimizer.step()

  0%|          | 0/200 [00:00<?, ?it/s]


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "D:\mlp\graph_neural_networks\venv\lib\site-packages\torch_geometric\utils\softmax.py", line 41, in softmax
    elif index is not None:
        N = maybe_num_nodes(index, num_nodes)
        src_max = scatter(src, index, dim, dim_size=N, reduce='max')
                  ~~~~~~~ <--- HERE
        src_max = src_max.index_select(dim, index)
        out = (src - src_max).exp()
  File "D:\mlp\graph_neural_networks\venv\lib\site-packages\torch_scatter\scatter.py", line 160, in scatter
        return scatter_min(src, index, dim, out, dim_size)[0]
    elif reduce == 'max':
        return scatter_max(src, index, dim, out, dim_size)[0]
               ~~~~~~~~~~~ <--- HERE
    else:
        raise ValueError
  File "D:\mlp\graph_neural_networks\venv\lib\site-packages\torch_scatter\scatter.py", line 72, in scatter_max
        out: Optional[torch.Tensor] = None,
       

In [64]:
data.edge_index

tensor([[ 163,  163,  163,  ..., 1887, 1902,  837],
        [ 402,  659, 1696,  ..., 2258, 1887, 1686]])

In [65]:
test_preds=  model(data)[~train_mask]
test_y=  data.y[~train_mask]

In [66]:
pr = test_preds.argmax(axis=1).numpy()

In [67]:
r = test_y.numpy()

In [68]:
print(classification_report(r, pr))

              precision    recall  f1-score   support

           0       0.77      0.78      0.77        77
           1       0.92      0.90      0.91       105
           2       0.83      0.87      0.85       214
           3       0.80      0.84      0.82       101
           4       0.85      0.75      0.80        44
           5       0.89      0.66      0.76        47
           6       0.74      0.75      0.74        89

    accuracy                           0.82       677
   macro avg       0.83      0.79      0.81       677
weighted avg       0.82      0.82      0.82       677

